In [5]:
import pandas as pd

import tensorflow as tf
import keras
import torch

import matplotlib.pyplot as plt
import seaborn as sb

In [11]:
df1 = pd.read_csv('../../../../../../data/r4.2/email.csv')


In [14]:
df1.shape

(2629979, 11)

In [5]:
df2 = dd.read_csv('email.csv')

In [7]:
df2.compute()

,id,date,user,pc,to,cc,bcc,from,size,attachments,content
0,{R3I7-S4TX96FG-8219JWFF},01/02/2010 07:11:45,LAP0338,PC-5758,Dean.Flynn.Hines@dtaa.com;Wade_Harrison@lockhe...,Nathaniel.Hunter.Heath@dtaa.com,NaN,Lynn.Adena.Pratt@dtaa.com,25830,0,middle f2 systems 4 july techniques powerful d...
1,{R0R9-E4GL59IK-2907OSWJ},01/02/2010 07:12:16,MOH0273,PC-6699,Odonnell-Gage@bellsouth.net,NaN,NaN,MOH68@optonline.net,29942,0,the breaking called allied reservations former...
2,{G2B2-A8XY58CP-2847ZJZL},01/02/2010 07:13:00,LAP0338,PC-5758,Penelope_Colon@netzero.com,NaN,NaN,Lynn_A_Pratt@earthlink.net,28780,0,slowly this uncinus winter beneath addition ex...
3,{A3A9-F4TH89AA-8318GFGK},01/02/2010 07:13:17,LAP0338,PC-5758,Judith_Hayden@comcast.net,NaN,NaN,Lynn_A_Pratt@earthlink.net,21907,0,400 other difficult land cirrocumulus powered ...
4,{E8B7-C8FZ88UF-2946RUQQ},01/02/2010 07:13:28,MOH0273,PC-6699,Bond-Raymond@verizon.net;Alea_Ferrell@msn.com;...,NaN,Odonnell-Gage@bellsouth.net,MOH68@optonline.net,17319,0,this kmh october holliswood number advised unu...
...,...,...,...,...,...,...,...,...,...,...,...
124903,{L7U1-S3KK01XW-3123TPUC},05/16/2011 20:54:18,HRL0540,PC-1117,Hadley.Willa.Hill@dtaa.com;Tarik.Linus.Hubbard...,Hedwig.Regina.Livingston@dtaa.com,NaN,Hedwig.Regina.Livingston@dtaa.com,33088,0,history designed stephen degree ignore them ad...
124904,{Q4Y3-X9HV77CL-3290SOZH},05/16/2011 20:54:43,LAF0991,PC-4973,Hu.Akeem.Vincent@dtaa.com,Brianna.Rebecca.Mcintyre@dtaa.com;Kermit.Coby....,NaN,Lucas.Ahmed.Ferrell@dtaa.com,33249,1,prince prince ahmose ahmose ankh prince prince...
124905,{L6B7-F7RU89CD-0355JGGE},05/16/2011 21:08:12,LAF0991,PC-4973,Doyle.Grant@netzero.com;HAV856@charter.net,NaN,NaN,Ferrell.Lucas@sbcglobal.net,41336,1,lifted documents 65 declined revival 14 unprec...
124906,{F4R9-W0OQ41HB-0157JSUE},05/16/2011 21:15:35,JMW0638,PC-1397,Eagan.Zephania.Talley@dtaa.com,NaN,NaN,Jonah.Merritt.Wilder@dtaa.com,39908,0,their official holmes face arranged among priz...


In [8]:
df3 = dd.read_csv('logon.csv')

In [9]:
df3.compute()

,id,date,user,pc,activity
0,{X1D9-S0ES98JV-5357PWMI},01/02/2010 06:49:00,NGF0157,PC-6056,Logon
1,{G2B3-L6EJ61GT-2222RKSO},01/02/2010 06:50:00,LRR0148,PC-4275,Logon
2,{U6Q3-U0WE70UA-3770UREL},01/02/2010 06:53:04,LRR0148,PC-4124,Logon
3,{I0N5-R7NA26TG-6263KNGM},01/02/2010 07:00:00,IRM0931,PC-7188,Logon
4,{D1S0-N6FH62BT-5398KANK},01/02/2010 07:00:00,MOH0273,PC-6699,Logon
...,...,...,...,...,...
854854,{J8R2-H0FA17MQ-9304VEVQ},05/17/2011 04:40:33,JDF0593,PC-6301,Logoff
854855,{P3L5-A2AC53JU-4012GIRD},05/17/2011 05:21:58,MLM0950,PC-9787,Logoff
854856,{P2S0-P1JS62IF-2767BEPH},05/17/2011 05:25:33,IRM0931,PC-7188,Logon
854857,{Z2U6-I8MA03NC-4963HKBH},05/17/2011 06:23:48,IRM0931,PC-7188,Logoff


In [10]:
df4 = dd.read_csv('device.csv')

In [11]:
df4.compute()

,id,date,user,pc,activity
0,{J1S3-L9UU75BQ-7790ATPL},01/02/2010 07:21:06,MOH0273,PC-6699,Connect
1,{N7B5-Y7BB27SI-2946PUJK},01/02/2010 07:37:41,MOH0273,PC-6699,Disconnect
2,{U1V9-Z7XT67KV-5649MYHI},01/02/2010 07:59:11,HPH0075,PC-2417,Connect
3,{H0Z7-E6GB57XZ-1603MOXD},01/02/2010 07:59:49,IIW0249,PC-0843,Connect
4,{L7P2-G4PX02RX-7999GYOY},01/02/2010 08:04:26,IIW0249,PC-0843,Disconnect
...,...,...,...,...,...
405375,{R7R7-Y9VH64MN-4427OTOU},05/16/2011 22:27:23,EIS0041,PC-0422,Disconnect
405376,{J1G6-G7KE64TX-7505AXXN},05/16/2011 22:43:49,IBB0359,PC-4176,Connect
405377,{I3V8-Q1KQ57JG-4571IXHJ},05/16/2011 22:48:39,IBB0359,PC-4176,Disconnect
405378,{W9Y8-O7VO98OA-0160JVWR},05/16/2011 23:22:29,IBB0359,PC-3620,Connect


In [12]:
df5 = dd.read_csv('file.csv')

In [13]:
df5.compute()

,id,date,user,pc,filename,content
0,{L9G8-J9QE34VM-2834VDPB},01/02/2010 07:23:14,MOH0273,PC-6699,EYPC9Y08.doc,D0-CF-11-E0-A1-B1-1A-E1 during difficulty over...
1,{H0W6-L4FG38XG-9897XTEN},01/02/2010 07:26:19,MOH0273,PC-6699,N3LTSU3O.pdf,25-50-44-46-2D carpenters 25 landed strait dis...
2,{M3Z0-O2KK89OX-5716MBIM},01/02/2010 08:12:03,HPH0075,PC-2417,D3D3WC9W.doc,D0-CF-11-E0-A1-B1-1A-E1 union 24 declined impo...
3,{E1I4-S4QS61TG-3652YHKR},01/02/2010 08:17:00,HPH0075,PC-2417,QCSW62YS.doc,D0-CF-11-E0-A1-B1-1A-E1 becoming period begin ...
4,{D4R7-E7JL45UX-0067XALT},01/02/2010 08:24:57,HSB0196,PC-8001,AU75JV6U.jpg,FF-D8
...,...,...,...,...,...,...
147883,{J2Q8-Z4UX02QT-2675RTJI},05/16/2011 23:22:31,IBB0359,PC-3620,R8DTDN2Q.txt,5A-44-4F-39 government operations led wound co...
147884,{S3G4-K5PB37EL-4663ADOI},05/16/2011 23:22:32,IBB0359,PC-3620,BLHCRL6W.pdf,25-50-44-46-2D throughout 20 once armed southe...
147885,{G6G8-X7JK94JA-2703OXCY},05/16/2011 23:22:33,IBB0359,PC-3620,ZHMDTTW0.doc,D0-CF-11-E0-A1-B1-1A-E1 companies river englan...
147886,{H8U9-F5GM78QB-9540DSQM},05/16/2011 23:22:33,IBB0359,PC-3620,AC0QL6KF.pdf,25-50-44-46-2D member august observed selected...


In [16]:
result = []
result2 = pd.DataFrame()

#total_len = 0
for chunk in df:
    toAppend = chunk.iloc[:, 1:3]
    result.append(toAppend)
    #total_len += len(chunk)
#print(total_len)
result2 = pd.concat(result)

In [17]:
result2.head

<bound method NDFrame.head of                         date     user
0        01/02/2010 07:11:45  LAP0338
1        01/02/2010 07:12:16  MOH0273
2        01/02/2010 07:13:00  LAP0338
3        01/02/2010 07:13:17  LAP0338
4        01/02/2010 07:13:28  MOH0273
...                      ...      ...
2629974  05/16/2011 20:54:18  HRL0540
2629975  05/16/2011 20:54:43  LAF0991
2629976  05/16/2011 21:08:12  LAF0991
2629977  05/16/2011 21:15:35  JMW0638
2629978  05/16/2011 21:16:26  BCM0653

[2629979 rows x 2 columns]>